In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import numpy as np
import tensorflow as tf
import json

In [3]:
with open('dataset-bpe.json') as fopen:
    data = json.load(fopen)

In [4]:
train_X = data['train_X']
train_Y = data['train_Y']
test_X = data['test_X']
test_Y = data['test_Y']

In [5]:
EOS = 2
GO = 1
vocab_size = 32000

In [6]:
train_Y = [i + [2] for i in train_Y]
test_Y = [i + [2] for i in test_Y]

In [8]:
from tensor2tensor.utils import beam_search

def pad_second_dim(x, desired_size):
    padding = tf.tile([[[0.0]]], tf.stack([tf.shape(x)[0], desired_size - tf.shape(x)[1], tf.shape(x)[2]], 0))
    return tf.concat([x, padding], 1)

class Translator:
    def __init__(self, size_layer, num_layers, embedded_size, learning_rate):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.GRUCell(size_layer,reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype = tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype = tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        embeddings = tf.Variable(tf.random_uniform([vocab_size, embedded_size], -1, 1))
        
        def forward(x, y, reuse = False):
            batch_size = tf.shape(x)[0]
            X_seq_len = tf.count_nonzero(x, 1, dtype = tf.int32)
            Y_seq_len = tf.count_nonzero(y, 1, dtype = tf.int32)
            with tf.variable_scope('model',reuse=reuse):
                encoder_embedded = tf.nn.embedding_lookup(embeddings, x)
                decoder_embedded = tf.nn.embedding_lookup(embeddings, y)
                rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
                last_output, last_state = tf.nn.dynamic_rnn(rnn_cells, encoder_embedded,
                                                  sequence_length=X_seq_len,
                                                  dtype = tf.float32)
                
            with tf.variable_scope("decoder",reuse=reuse):
                
                attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units = size_layer, 
                                                                    memory = last_output)
                rnn_cells = tf.contrib.seq2seq.AttentionWrapper(
                    cell = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)]), 
                    attention_mechanism = attention_mechanism,
                    attention_layer_size = size_layer)
                
                initial_state = rnn_cells.zero_state(batch_size, tf.float32).clone(cell_state=last_state)
                outputs, _ = tf.nn.dynamic_rnn(rnn_cells, decoder_embedded, 
                                               sequence_length=Y_seq_len,
                                               initial_state = initial_state,
                                               dtype = tf.float32)
                
                return tf.layers.dense(outputs,vocab_size)
            
        main = tf.strided_slice(self.X, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        self.training_logits = forward(self.X, decoder_input, reuse = False)
        
        self.training_logits = self.training_logits[:, :tf.reduce_max(self.Y_seq_len)]
        self.training_logits = pad_second_dim(self.training_logits, tf.reduce_max(self.Y_seq_len))
            
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        initial_ids = tf.fill([batch_size], GO)
        def symbols_to_logits(ids):
            x = tf.contrib.seq2seq.tile_batch(self.X, 1)
            logits = forward(x, ids, reuse = True)
            return logits[:, tf.shape(ids)[1]-1, :]
        
        final_ids, final_probs, _ = beam_search.beam_search(
            symbols_to_logits,
            initial_ids,
            1,
            tf.reduce_max(self.X_seq_len),
            vocab_size,
            0.0,
            eos_id = EOS)
        
        self.fast_result = final_ids

In [9]:
size_layer = 512
num_layers = 2
embedded_size = 256
learning_rate = 1e-3
batch_size = 128
epoch = 20

In [10]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(size_layer, num_layers, embedded_size, learning_rate)
sess.run(tf.global_variables_initializer())

Instructions for updating:
reduction_indices is deprecated, use axis instead
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/

In [11]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [12]:
batch_x = pad_sequences(train_X[:10], padding='post')
batch_y = pad_sequences(train_Y[:10], padding='post')

sess.run([model.fast_result, model.cost, model.accuracy], 
         feed_dict = {model.X: batch_x, model.Y: batch_y})

[array([[[    1, 13872, 13872, 31767, 11095, 11095, 11095, 20258, 20258,
          20258, 20258,  6960, 11142, 20354, 20354, 20354, 20354, 31683,
          31683, 26948,  5179,  4466,  4466,  4466,  4466,  5385,  5385,
           5385, 18081, 18081, 18081,  7270, 14117, 27127, 29460, 19935,
           6996]],
 
        [[    1, 21436,  9561,  9561,  7197,  9561,  7197,  5339, 26032,
           5339,  4247, 21722,  4247, 31038, 26507, 26507, 26507, 26507,
          26507, 15165, 15165, 15165, 22856, 22856, 24925, 24925, 17107,
          17107, 13494, 21013, 21013, 21013, 17107,  5741,  5741,  5741,
          29063]],
 
        [[    1, 18442, 23238, 23238, 25687, 25687,  1263,  1263, 13867,
          13867, 21591, 21591, 21591, 13440, 13440, 14704, 14704, 16520,
          16520,  1165,  1165, 20886, 20886, 20886, 26485, 13691, 13691,
          13691, 13691, 17436, 17436, 11480, 11480, 11480, 24671, 19593,
           3945]],
 
        [[    1,  6987,  6987, 15750, 13615, 13615, 13615, 13

In [13]:
import tqdm

for e in range(epoch):
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = pad_sequences(train_X[i : index], padding='post')
        batch_y = pad_sequences(train_Y[i : index], padding='post')
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = pad_sequences(test_X[i : index], padding='post')
        batch_y = pad_sequences(test_Y[i : index], padding='post')
        feed = {model.X: batch_x,
                model.Y: batch_y,}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('epoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 1, training avg loss 6.788980, training avg acc 0.123003
epoch 1, testing avg loss 6.129451, testing avg acc 0.149149


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 2, training avg loss 5.775488, training avg acc 0.158012
epoch 2, testing avg loss 5.788449, testing avg acc 0.158840


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

...

epoch 18, training avg loss 3.945296, training avg acc 0.248735
epoch 18, testing avg loss 6.194018, testing avg acc 0.153860


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 19, training avg loss 3.922567, training avg acc 0.250986
epoch 19, testing avg loss 6.230792, testing avg acc 0.154232


minibatch loop: 100%|██████████| 40/40 [00:06<00:00,  6.40it/s, accuracy=0.134, cost=6.04]

epoch 20, training avg loss 4.049239, training avg acc 0.237949
epoch 20, testing avg loss 6.154497, testing avg acc 0.153501


In [14]:
from tensor2tensor.utils import bleu_hook

In [15]:
results = []
for i in tqdm.tqdm(range(0, len(test_X), batch_size)):
    index = min(i + batch_size, len(test_X))
    batch_x = pad_sequences(test_X[i : index], padding='post')
    feed = {model.X: batch_x}
    p = sess.run(model.fast_result,feed_dict = feed)[:,0,:]
    result = []
    for row in p:
        result.append([i for i in row if i > 3])
    results.extend(result)

100%|██████████| 40/40 [02:43<00:00,  4.09s/it]


In [16]:
rights = []
for r in test_Y:
    rights.append([i for i in r if i > 3])

In [17]:
bleu_hook.compute_bleu(reference_corpus = rights,
                       translation_corpus = results)

0.025584696